# Example 7: Reduction and Speed Analysis

After building a circuit, one might want to do some stuff to **reduce the
hardware into simpler nets** as well as **analyze various metrics of the
hardware**. This functionality is provided in the Passes part of PyRTL
and will demonstrated here.

In [28]:
import pyrtl
from pyrtl.analysis import estimate

## Part 1: Timing Analysis

**Timing and area usage** are key considerations of any hardware block that one
makes.

PyRTL provides functions to do these opertions

In [29]:
# Creating a sample harware block
pyrtl.reset_working_block()
const_wire = pyrtl.Const(6, bitwidth=4)
in_wire2 = pyrtl.Input(bitwidth=4, name="input2")
out_wire = pyrtl.Output(bitwidth=5, name="output")
out_wire <<= const_wire + in_wire2

### Now we will do the timing analysis as well as print out the critical path

In [30]:
# Generating timing analysis information
print("Pre Synthesis:")
timing = estimate.TimingAnalysis()
timing.print_max_length()

Pre Synthesis:
The total block timing delay is  386.9


We are also able to **print out the critical paths** as well as get them
back as an array.

In [31]:
critical_path_info = timing.critical_path()

Critical path 0 :
   The first wire is: const_60_6/4C
   tmp234/5W <-- + -- const_60_6/4C, input2/4I 

Critical path 1 :
   The first wire is: input2/4I
   tmp234/5W <-- + -- const_60_6/4C, input2/4I 

Critical path 2 :
   The first wire is: const_60_6/4C
   tmp234/5W <-- + -- const_60_6/4C, input2/4I 
   output/5O <-- w -- tmp234/5W 

Critical path 3 :
   The first wire is: input2/4I
   tmp234/5W <-- + -- const_60_6/4C, input2/4I 
   output/5O <-- w -- tmp234/5W 



## Part 2: Area Analysis

PyRTL also provides **estimates for the area** that would be used up if the
circuit was printed as an **ASIC**

In [32]:
logic_area, mem_area = estimate.area_estimation(tech_in_nm=65)
est_area = logic_area + mem_area
print("Estimated Area of block", est_area, "sq mm")
print()

Estimated Area of block 4.870008e-05 sq mm



## Part 3: Synthesis

Synthesis is the operation of **reducing the circuit into simpler components**
The base synthesis function breaks down the more complex logic operations
into logic gates (keeps registers and memories intact) as well as **reduces
all combinatorial logic into ops that only use one bitwidth wires**

This synthesis allows for PyRTL to make **optimizations to the net structure**
as well as prepares it for further transformations on the PyRTL Toolchain

In [33]:
pyrtl.synthesize()

print("Pre Optimization:")
timing = estimate.TimingAnalysis()
timing.print_max_length()
for net in pyrtl.working_block().logic:
    print(str(net))
print()

Pre Optimization:
The total block timing delay is  1236.3999999999996
tmp235_synth_0/1W <-- w -- const_65_0/1C 
tmp277_synth_2/1W <-- w -- tmp276_synth_1/1W 
tmp_output_synth_3/1W <-- w -- tmp234_synth_3/1W 
tmp277_synth_0/1W <-- w -- tmp238_synth_0/1W 
tmp303/1W <-- & -- tmp235_synth_0/1W, tmp236_synth_0/1W 
tmp282/1W <-- s -- input2/4I ((3,))
tmp234_synth_0/1W <-- w -- tmp278_synth_0/1W 
tmp275_synth_1/1W <-- w -- tmp269_synth_0/1W 
tmp277_synth_1/1W <-- w -- tmp276_synth_0/1W 
tmp246_synth_0/1W <-- w -- tmp244_synth_0/1W 
tmp_input2_synth_0/1W <-- w -- tmp279/1W 
tmp268_synth_0/1W <-- w -- tmp297/1W 
tmp311/1W <-- & -- tmp258_synth_0/1W, tmp254_synth_0/1W 
tmp258_synth_0/1W <-- w -- tmp256_synth_0/1W 
tmp259_synth_0/1W <-- w -- tmp291/1W 
tmp236_synth_0/1W <-- w -- tmp_input2_synth_0/1W 
tmp271_synth_0/1W <-- w -- tmp286/1W 
tmp234_synth_1/1W <-- w -- tmp278_synth_1/1W 
tmp234_synth_3/1W <-- w -- tmp278_synth_3/1W 
tmp265_synth_0/1W <-- w -- tmp308/1W 
tmp254_synth_0/1W <-- w -- tmp

## Part 4: Optimization

PyRTL has functions built in to **eliminate unnecessary logic** from the
circuit.

These functions are all done with a simple call:

In [34]:
pyrtl.optimize()

PyrtlError: need graphviz installed (try "pip install graphviz")

### Now to see the difference

In [35]:
print("Post Optimization:")
timing = estimate.TimingAnalysis()
timing.print_max_length()

Post Optimization:
The total block timing delay is  345.66999999999996


In [36]:
for net in pyrtl.working_block().logic:
    print(str(net))

tmp311/1W <-- & -- tmp281/1W, tmp280/1W 
tmp296/1W <-- ~ -- tmp280/1W 
tmp291/1W <-- ~ -- tmp281/1W 
output/5O <-- w -- tmp283/5W 
tmp307/1W <-- ^ -- tmp291/1W, tmp280/1W 
tmp301/1W <-- ^ -- tmp282/1W, tmp308/1W 
tmp308/1W <-- | -- tmp298/1W, tmp311/1W 
tmp282/1W <-- s -- input2/4I ((3,))
tmp298/1W <-- | -- tmp281/1W, tmp280/1W 
tmp284/1W <-- & -- tmp282/1W, tmp308/1W 
tmp279/1W <-- s -- input2/4I ((0,))
tmp283/5W <-- c -- tmp284/1W, tmp301/1W, tmp307/1W, tmp296/1W, tmp279/1W 
tmp281/1W <-- s -- input2/4I ((2,))
tmp280/1W <-- s -- input2/4I ((1,))


### As we can see, the number of nets in the circuit were drastically reduced by the optimization algorithm.